In [60]:

import warnings
from collections import defaultdict
from datetime import datetime
import distutils
import pandas as pd
from tqdm import tqdm
import numpy as np
from pandas.api.types import is_datetime64_any_dtype as is_datetime

In [61]:

X = pd.read_csv("../data/processed/gw_stage2.CSV",
                    parse_dates=['Datetime'],
                    index_col=['Datetime'])
print('Initial Input Size: ',X.shape)
y = pd.read_csv("../data/processed/spring.CSV",
                    parse_dates=['datetime'],
                    index_col=['datetime'])

X.head(10)

key_value = np.loadtxt("../data/processed/NameKey2.CSV", dtype= "str", delimiter=",", skiprows=1)
stationNames = { k:v for k,v in key_value }

Initial Input Size:  (1101, 74)


In [62]:
#Prep Dict Keys as an int and a string
keyint=list(range(0,74))
#print(keyint)
key_str=[str(x) for x in keyint]


In [63]:
#Set up new Df
new_df=pd.DataFrame(columns=['Station_Name', "key", 'ts_type',"start_timestamp", 'frequency', 'h_ts', 'series_length'])
new_df

,Station_Name,key,ts_type,start_timestamp,frequency,h_ts,series_length


In [64]:
lst_dict=[]
for i in range(74):
    if i<=29:
        station_type='GW'
    else:
        station_type='SW'
    a=stationNames[key_str[i]]
    #print(a)
    test=X.loc[:,a]
    z=test.tolist()
    lst_dict.append({'Station_Name': test.name, 'key':key_str[i], 'ts_type': station_type, 
                       'start_timestamp':"2017-01-27", 'frequency':"1D", 'h_ts':z, 
                       'series_length': test.size})
    df2=new_df.append(lst_dict)
    
    

In [65]:
z=y['Gage_Height'].tolist()
spring_dict=[{'Station_Name': "Wakulla_Springs_USGS", 'key':74, 'ts_type': "spring", 
                       'start_timestamp':"2017-01-27", 'frequency':"1D", 'h_ts': z, 
                       'series_length': len(z)}]
df2=df2.append(spring_dict, ignore_index=True)
df2

,Station_Name,key,ts_type,start_timestamp,frequency,h_ts,series_length
0,USGS-LESTER LEWIS/S788,0,GW,2017-01-27,1D,"[6.340833333, 6.220729167, 6.1534375, 6.046041...",1101
1,NWFWMD-BIKE TRAIL / NITRATE POT. MAP/S795,1,GW,2017-01-27,1D,"[15.02541667, 15.08229167, 15.1109375, 15.0711...",1101
2,NWFWMD-BRADFORD BROOK DEEP/S793,2,GW,2017-01-27,1D,"[20.72239583, 20.79333333, 20.8790625, 20.8895...",1101
3,NWFWMD-BRADFORD BROOK SHALLOW,3,GW,2017-01-27,1D,"[38.61302083, 38.61760417, 38.6271875, 38.5586...",1101
4,TOM BROWN TEST/S791,4,GW,2017-01-27,1D,"[25.915625, 26.0184375, 26.03791667, 25.949375...",1101
...,...,...,...,...,...,...,...
70,LAKE KILLARNY AT MCLAUGHLIN DRIVE,70,SW,2017-01-27,1D,"[73.27979167, 73.21723959, 73.16114583, 73.118...",1101
71,LAKE KANTURK AT CLIFDEN DR,71,SW,2017-01-27,1D,"[73.31546875, 73.25291667, 73.19697917, 73.152...",1101
72,INDIAN SPRING RUN AT SR61,72,SW,2017-01-27,1D,"[5.243125, 5.170104167, 5.105208333, 5.0539583...",1101
73,ST. MARKS RIVER - UPPER DISCHARGE BEFORE SWALLET,73,SW,2017-01-27,1D,"[9.488892223, 9.488892223, 9.488892223, 9.4888...",1101


In [35]:
def write_compact_to_ts(
    df: pd.DataFrame,
    filename: str,
    static_columns: list,
    time_varying_columns: list,
    sep: str = ";",
    encoding: str = "utf-8",
    date_format: str = "%Y-%m-%d %H-%M-%S",
    chunk_size: int = 50,
):
    """Writes a dataframe in the compact form to disk
    Args:
        df (pd.DataFrame): The dataframe in compact form
        filename (str): Filename to which the dataframe should be written to
        static_columns (list): List of column names of static features
        time_varying_columns (list): List of column names of time varying columns
        sep (str, optional): Separator with which the arrays are stored in the text file. Defaults to ";".
        encoding (str, optional): encoding of the text file. Defaults to "utf-8".
        date_format (str, optional): Format in which datetime shud be written out in text file. Defaults to "%Y-%m-%d %H-%M-%S".
        chunk_size (int, optional): Chunk size while writing files to disk. Defaults to 50.
    Returns:
        None
    """
    if sep == ":":
        warnings.warn(
            "Using `:` as separator will not work well if `:` is present in the string representation of date time."
        )
    with open(filename, "w", encoding=encoding) as f:
        for c, dtype in df.dtypes.items():
            if c in static_columns:
                typ = "static"
            elif c in time_varying_columns:
                typ = "time_varying"
            f.write(f"@column {c} {dtype.name} {typ}")
            f.write("\n")
        f.write(f"@data")
        f.write("\n")

        def write_ts(x):
            l = ""
            for c in x.index:
                if isinstance(x[c], np.ndarray):
                    l += "|".join(x[c].astype(str))
                    l += sep
                elif isinstance(x[c], pd.Timestamp):
                    l += x[c].strftime(date_format)
                    l += sep
                else:
                    l += str(x[c])
                    l += sep
            l += "\n"
            return l

        [
            f.writelines([write_ts(x.loc[i]) for i in tqdm(x.index)])
            for x in tqdm(
                np.split(df, np.arange(chunk_size, len(df), chunk_size)),
                desc="Writing in Chunks...",
            )
        ]

In [36]:
write_compact_to_ts(df2,
    static_columns = ['Station_Name', 'key', 'ts_type','series_length', 'frequency', 'start_timestamp'], 
    time_varying_columns = ['h_ts'],
    filename=f"../data/processed/GW_SW_Daily.ts",
    sep=";")

100%|█████████████████████████████████████████████████████████████████████████████████| 50/50 [00:00<00:00, 847.72it/s]

Writing in Chunks...: 100%|██████████████████████████████████████████████████████████████| 2/2 [00:00<00:00, 20.41it/s]


In [66]:
def compact_to_expanded(
    df, timeseries_col, static_cols, time_varying_cols, ts_identifier):
    def preprocess_expanded(x):
        ### Fill missing dates with NaN ###
        # Create a date range from  start
        dr = pd.date_range(
            start=x["start_timestamp"],
            periods=len(x["h_ts"]),
            freq=x["frequency"],
        )
        df_columns = defaultdict(list)
        df_columns["timestamp"] = dr
        for col in [ts_identifier, timeseries_col] + static_cols + time_varying_cols:
            df_columns[col] = x[col]
        return pd.DataFrame(df_columns)
        

    all_series = []
    for i in tqdm(range(len(df))):
        all_series.append(preprocess_expanded(df.iloc[i]))
    df = pd.concat(all_series)
    del all_series
    return df

In [78]:
exp_df=compact_to_expanded(
    df2, timeseries_col='h_ts',
    static_cols=['key', 'ts_type','series_length', 'frequency', 'start_timestamp'],
    time_varying_cols=[],
    ts_identifier='Station_Name')

exp_df

100%|█████████████████████████████████████████████████████████████████████████████████| 75/75 [00:00<00:00, 765.55it/s]


,timestamp,Station_Name,h_ts,key,ts_type,series_length,frequency,start_timestamp
0,2017-01-27,USGS-LESTER LEWIS/S788,6.340833,0,GW,1101,1D,2017-01-27
1,2017-01-28,USGS-LESTER LEWIS/S788,6.220729,0,GW,1101,1D,2017-01-27
2,2017-01-29,USGS-LESTER LEWIS/S788,6.153437,0,GW,1101,1D,2017-01-27
3,2017-01-30,USGS-LESTER LEWIS/S788,6.046042,0,GW,1101,1D,2017-01-27
4,2017-01-31,USGS-LESTER LEWIS/S788,5.991562,0,GW,1101,1D,2017-01-27
...,...,...,...,...,...,...,...,...
1096,2020-01-28,Wakulla_Springs_USGS,4.814896,74,spring,1101,1D,2017-01-27
1097,2020-01-29,Wakulla_Springs_USGS,4.811146,74,spring,1101,1D,2017-01-27
1098,2020-01-30,Wakulla_Springs_USGS,4.807292,74,spring,1101,1D,2017-01-27
1099,2020-01-31,Wakulla_Springs_USGS,4.777188,74,spring,1101,1D,2017-01-27


In [73]:
idx = pd.date_range("2017-01-27", periods=1101, freq="D")
ts = pd.Series(range(len(idx)), index=idx)

AttributeError: 'DataFrame' object has no attribute 'timestamp'

In [51]:
def reduce_memory_footprint(df):
    dtypes = df.dtypes
    object_cols = dtypes[dtypes == "object"].index.tolist()
    float_cols = dtypes[dtypes == "float64"].index.tolist()
    int_cols = dtypes[dtypes == "int64"].index.tolist()
    df[int_cols] = df[int_cols].astype("int32")
    df[object_cols] = df[object_cols].astype("category")
    df[float_cols] = df[float_cols].astype("float32")
    return df


In [55]:
exp_df.info(memory_usage="deep", verbose=False)
exp_df = reduce_memory_footprint(exp_df)

exp_df.info(memory_usage="deep", verbose=False)

<class 'pandas.core.frame.DataFrame'>
Int64Index: 82575 entries, 0 to 1100
Columns: 7 entries, timestamp to frequency
dtypes: datetime64[ns](1), float64(1), int64(1), object(4)
memory usage: 23.1 MB
<class 'pandas.core.frame.DataFrame'>
Int64Index: 82575 entries, 0 to 1100
Columns: 7 entries, timestamp to frequency
dtypes: category(4), datetime64[ns](1), float32(1), int32(1)
memory usage: 2.2 MB


In [79]:
### Train/Test Split of 70% Train 20% Val and 20% Test. Dates for these are calcualted outside of

test_mask =(exp_df['timestamp'] >= '2019-03-08')

train = exp_df[~(test_mask)]

test = exp_df[test_mask]


In [81]:
train.to_csv("../data/processed/daily_train.CSV")
#val.to_csv("../data/processed/daily_val.CSV")
test.to_csv("../data/processed/daily_test.CSV")